**Задача:**
- сделать классификацию тональности текста с помощью BERT
- дообучить модель на предоставленном датасете
- добавить оценку качества модели на валидационном датасете после каждой эпохи обучения средствами библиотеки Evaluate (F1)
- сохранять веса модели после каждой эпохи обучения
- сохранить веса лучшей модели

# Imports & Const

In [ ]:
!pip install -q datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
from typing import List, Tuple

In [ ]:
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [ ]:
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
MINIBATCH_SIZE = 64

# Initialization & Tokenize

In [ ]:
# инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# загрузка датасета
df = load_dataset(DATASET_NAME, DATASET_LANGUAGE)

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# удалим ненужное
df = df.remove_columns('index_id')
df

DatasetDict({
    train: Dataset({
        features: ['category', 'text'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['category', 'text'],
        num_rows: 99
    })
    test: Dataset({
        features: ['category', 'text'],
        num_rows: 204
    })
})

In [ ]:
# токенизация обучающего и валидационного датасетов
tokenized_data = df.map(lambda token: tokenizer(token['text'], truncation=False), batched=True)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [ ]:
encode_categories = tokenized_data.class_encode_column('category')
tokenized_data['train'] = tokenized_data['train'].add_column(name='labels', column=encode_categories['train']['category'])
tokenized_data['validation'] = tokenized_data['validation'].add_column(name='labels', column=encode_categories['validation']['category'])
tokenized_data['test'] = tokenized_data['test'].add_column(name='labels', column=encode_categories['test']['category'])

Casting to class labels:   0%|          | 0/701 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/99 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/204 [00:00<?, ? examples/s]

In [ ]:
labels = encode_categories['train'].features['category'].names

In [ ]:
# инициализация метрики из evaluate
f1_score = evaluate.load("f1")

def compute_f1(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_score.compute(predictions=predictions, references=labels, average='macro')

In [ ]:
# создание дата коллатора
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Train

In [ ]:
# инициализация классификатора через AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for param in model.parameters():
    param.data = param.data.contiguous()

In [ ]:
# задаём аргументы для обучения через TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [ ]:
# создаём Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=data_collator,
    compute_metrics=compute_f1,
    tokenizer=tokenizer,
)

In [ ]:
# обучаем
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.740479,0.191387
2,No log,1.336900,0.491690
3,No log,0.934099,0.779055
4,No log,0.699311,0.780766
5,No log,0.546815,0.828426
6,No log,0.509709,0.837945
7,No log,0.484475,0.842266
8,No log,0.468266,0.833802
9,No log,0.461625,0.836818
10,No log,0.491485,0.823517


TrainOutput(global_step=165, training_loss=0.45990919633345173, metrics={'train_runtime': 8394.4233, 'train_samples_per_second': 1.253, 'train_steps_per_second': 0.02, 'total_flos': 317085667097400.0, 'train_loss': 0.45990919633345173, 'epoch': 15.0})

# Metrics & Results

In [ ]:
# создаём пайплайн классификации текста для получения предсказаний
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
# получаем предсказания на валидационной выборке и формируем classification_report
val_predictions = trainer.predict(tokenized_data['validation'])
val_preds = np.argmax(val_predictions.predictions, axis=-1)
val_report = classification_report(tokenized_data['validation']['labels'], val_preds, target_names=labels)

In [ ]:
print(val_report)

                    precision    recall  f1-score   support

     entertainment       0.86      0.67      0.75         9
         geography       0.75      0.75      0.75         8
            health       1.00      0.82      0.90        11
          politics       0.88      1.00      0.93        14
science/technology       0.89      0.96      0.92        25
            sports       1.00      0.92      0.96        12
            travel       0.67      0.70      0.68        20

          accuracy                           0.85        99
         macro avg       0.86      0.83      0.84        99
      weighted avg       0.85      0.85      0.85        99



In [ ]:
# получаем предсказания на тестовой выборке и формируем classification_report
test_predictions = trainer.predict(tokenized_data['test'])
test_preds = np.argmax(test_predictions.predictions, axis=-1)
test_report = classification_report(tokenized_data['test']['labels'], test_preds, target_names=labels)


In [ ]:
print(test_report)

                    precision    recall  f1-score   support

     entertainment       0.92      0.58      0.71        19
         geography       0.89      0.94      0.91        17
            health       1.00      0.86      0.93        22
          politics       0.97      1.00      0.98        30
science/technology       0.85      0.92      0.89        51
            sports       0.96      0.96      0.96        25
            travel       0.86      0.95      0.90        40

          accuracy                           0.91       204
         macro avg       0.92      0.89      0.90       204
      weighted avg       0.91      0.91      0.90       204



In [ ]:
# пишем вывод
'''
В отличие от log_regression результаты гораздо лучше, на тестовой выборке мы видим f1_score ~0.9 и если немного поработать с параметрами,
то можно добится результата ~0.97 по f1_score, хотя обучение данной модели занимает гораздо больше времени, учитавая, что модель обучается на CPU
'''